## Lecture 14 of CMU16-745 (2025)

#### py_rbsim.ipynb

In [2]:
import numpy as np


In [59]:
def hat(v):
    return np.array([[0,-v[2],v[1]],
                     [v[2],0,-v[0]],
                     [-v[1],v[0],0]])


In [127]:
def L(q):
    global s,v,L_u,L_l
    s = q[0]
    v = q[1:]
    L_u = np.hstack((np.expand_dims(s,axis=0),-v.T))
    L_l = np.hstack((v,s*np.eye(3) + hat(v.flatten())))
    return np.vstack((L_u,L_l))

In [128]:
def R(q):
    s = q[0]
    v = q[1:]
    R_u = np.hstack((np.expand_dims(s,axis=0),-v.T))
    R_l = np.hstack((v,s*np.eye(3) - hat(v.flatten())))
    return np.vstack((R_u,R_l))

In [8]:
T = np.diagflat([1,-1,-1,-1])
H = np.vstack((np.zeros((1,3)),np.eye(3)))

In [10]:
def G(q):
    return L(q)@H

def Q(q):
    return H.T@(R(q).T@L(q))@H

In [11]:
J = np.diagflat([1,2,3])
h = 0.1

In [15]:
def vec(Mat):
    global M_global
    M_global = Mat
    m, n = np.shape(Mat)[0], np.shape(Mat)[1]
    return np.reshape(Mat,(m*n,1), order='F')

In [97]:
#initial conditions
Q0 = np.eye(3)
q0 = np.array([1,0,0,0]).reshape(4,1)
# w0 = np.array([1,-1,0.5]).reshape(3,1) # test
w0 = 2*np.random.rand(3,1)-1.0
x0 = np.vstack((vec(Q0),w0))
x0q = np.vstack((q0,w0))


In [98]:
# dynamics
def dynamics(x):
    global Qm, om, Qd, omd
    Qm = np.reshape(x[0:9],(3,3)).T
    om = x[9:]
    Qd = Qm@hat(om.flatten())
    omd = np.linalg.solve(-J,hat(om.flatten())@J@om).reshape(3,1)

    xd = np.vstack((vec(Qd), omd))
    return xd

In [103]:
def rkstep(x):
    # global f1, f2, f3, f4
    f1 = dynamics(x)
    f2 = dynamics(x + 0.5*h*f1)
    f3 = dynamics(x + 0.5*h*f2)
    f4 = dynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    return xn

In [99]:
xk = x0
for k in range(10000):
    xk = rkstep(xk)
    # print(xk)

In [100]:
Qk = np.reshape(xk[0:9],(3,3)).T

In [101]:
Qk.T@Qk

array([[ 9.99981970e-01, -1.76396813e-06,  3.12012650e-06],
       [-1.76396813e-06,  9.99985764e-01, -7.63808019e-06],
       [ 3.12012650e-06, -7.63808019e-06,  9.99994322e-01]])

In [102]:
# Quaternion dynamics
def qdynamics(x):
    q = x[0:4]
    w = x[4:7]
    qd = 0.5*L(q)@H@w
    wd = np.linalg.solve(-J,hat(w.flatten())@J@w).reshape(3,1)
    xd = np.vstack((qd, wd))
    return xd

In [106]:
def qrkstep(x):
    # global f1, f2, f3, f4
    f1 = qdynamics(x)
    f2 = qdynamics(x + 0.5*h*f1)
    f3 = qdynamics(x + 0.5*h*f2)
    f4 = qdynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[0:4] = xn[0:4]/np.linalg.norm(xn[0:4])
    return xn

In [129]:
xkq = x0q
for k in range(10000):
    xkq = qrkstep(xkq)

In [130]:
qk = xkq[0:4]
qk

array([[-0.49001482],
       [ 0.22964763],
       [-0.39549734],
       [ 0.74211138]])

In [131]:
np.linalg.norm(qk)

np.float64(1.0)

In [132]:
Q(qk).T@Q(qk)

array([[ 1.00000000e+00, -1.38777878e-17, -3.12250226e-17],
       [-1.38777878e-17,  1.00000000e+00, -1.11022302e-16],
       [-3.12250226e-17, -1.11022302e-16,  1.00000000e+00]])